In [1]:
from cpm.generators import Parameters, Wrapper

# import components we want to use

# import components we want to use
from cpm.models.utils import Nominal
from cpm.models.learning import DeltaRule

# import some useful stuff
from prettyformatter import pprint as pp  ## pretty print
import numpy as np

In [2]:
## create a single trial as a dictionary
trial = {
    "trials": np.array([1, 2]),
    "feedback": np.array([1]),
    "attention": np.array([1, 0]),  ## not used in the model
    "misc": np.array([1, 0]),  ## not used in the model
}

## create a set of parameters
## the parameter class is only used for the Wrapper
## other classes and modules will convert the parameters to this class automatically
parameters = Parameters(
    alpha=0.1, temperature=1, values=np.array([[-0.5, 0.7, 0.9, 0]])
)

In [3]:
def model(parameters, trial):
    ## import parameters
    alpha = parameters.alpha
    temperature = parameters.temperature
    ## import variables
    weights = parameters.values

    ## import trial
    stimulus = trial.get("trials")
    stimulus = Nominal(target=stimulus, bits=4)
    feedback = trial.get("feedback")

    ## specify the computations for a given trial
    ## multiply weights with stimulis vector
    active = stimulus * weights
    ## this is a mock-up policy
    policy = np.sum(active) * temperature
    ## learning
    learning = DeltaRule(weights=active, feedback=feedback, alpha=alpha, input=stimulus)
    learning.compute()
    error = learning.weights
    weights += error
    output = {
        "policy": np.array([policy]),
        "values": weights,
        "dependent": np.array([policy]),
    }
    return output

In [4]:
model(parameters, trial)

{'policy': array([0.2]),
 'values': array([[-0.42,  0.78,  0.9 ,  0.  ]]),
 'dependent': array([0.2])}

In [5]:
data = {
    "trials": np.array(
        [[2, 3], [1, 4], [3, 2], [4, 1], [2, 3], [2, 3], [1, 4], [3, 2], [4, 1], [2, 3]]
    ),
    "feedback": np.array([[1], [0], [1], [0], [1], [1], [0], [1], [0], [1]]),
}

In [6]:
# Add new parameters to the model
params = {
    "alpha": 0.5,
    "temperature": 1,
    "values": np.zeros((1, 4)),
}

params = Parameters(**params)

wrap = Wrapper(model=model, parameters=params, data=data)
wrap.run()
wrap.dependent

array([[0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.]])

In [7]:
pp(wrap.export())

   policy_0  values_0  values_1  values_2  values_3  dependent_0  ppt
0       0.0       0.0       0.5       0.5       0.0          0.0    0
0       0.0       0.0       0.5       0.5       0.0          0.0    0
0       1.0       0.0       0.5       0.5       0.0          1.0    0
0       0.0       0.0       0.5       0.5       0.0          0.0    0
0       1.0       0.0       0.5       0.5       0.0          1.0    0
0       1.0       0.0       0.5       0.5       0.0          1.0    0
0       0.0       0.0       0.5       0.5       0.0          0.0    0
0       1.0       0.0       0.5       0.5       0.0          1.0    0
0       0.0       0.0       0.5       0.5       0.0          0.0    0
0       1.0       0.0       0.5       0.5       0.0          1.0    0


In [8]:
wrap.reset(parameters={"alpha": 0.05, "temperature": 1, "values": np.zeros((1, 4))})
wrap.run()
pp(wrap.export())

   policy_0  values_0  values_1  values_2  values_3  dependent_0  ppt
0   0.00000       0.0  0.050000  0.050000       0.0      0.00000    0
0   0.00000       0.0  0.050000  0.050000       0.0      0.00000    0
0   0.10000       0.0  0.095000  0.095000       0.0      0.10000    0
0   0.00000       0.0  0.095000  0.095000       0.0      0.00000    0
0   0.19000       0.0  0.135500  0.135500       0.0      0.19000    0
0   0.27100       0.0  0.171950  0.171950       0.0      0.27100    0
0   0.00000       0.0  0.171950  0.171950       0.0      0.00000    0
0   0.34390       0.0  0.204755  0.204755       0.0      0.34390    0
0   0.00000       0.0  0.204755  0.204755       0.0      0.00000    0
0   0.40951       0.0  0.234280  0.234280       0.0      0.40951    0


In [9]:
wrap.simulation

[{'policy': array([0.]),
  'values': array([[0.  , 0.05, 0.05, 0.  ]]),
  'dependent': array([0.])},
 {'policy': array([0.]),
  'values': array([[0.  , 0.05, 0.05, 0.  ]]),
  'dependent': array([0.])},
 {'policy': array([0.1]),
  'values': array([[0.   , 0.095, 0.095, 0.   ]]),
  'dependent': array([0.1])},
 {'policy': array([0.]),
  'values': array([[0.   , 0.095, 0.095, 0.   ]]),
  'dependent': array([0.])},
 {'policy': array([0.19]),
  'values': array([[0.    , 0.1355, 0.1355, 0.    ]]),
  'dependent': array([0.19])},
 {'policy': array([0.271]),
  'values': array([[0.     , 0.17195, 0.17195, 0.     ]]),
  'dependent': array([0.271])},
 {'policy': array([0.]),
  'values': array([[0.     , 0.17195, 0.17195, 0.     ]]),
  'dependent': array([0.])},
 {'policy': array([0.3439]),
  'values': array([[0.      , 0.204755, 0.204755, 0.      ]]),
  'dependent': array([0.3439])},
 {'policy': array([0.]),
  'values': array([[0.      , 0.204755, 0.204755, 0.      ]]),
  'dependent': array([0.])},

In [10]:
wrap.save(filename="test")

In [11]:
experiment = []
for i in range(100):
    ppt = {
        "ppt": i,
        "trials": np.array(
            [
                [2, 3],
                [1, 4],
                [3, 2],
                [4, 1],
                [2, 3],
                [2, 3],
                [1, 4],
                [3, 2],
                [4, 1],
                [2, 3],
            ]
        ),
        "feedback": np.array([[1], [0], [1], [0], [1], [1], [0], [1], [0], [1]]),
        "observed": np.array([[1], [0], [1], [0], [1], [1], [0], [1], [0], [1]]),
    }
    experiment.append(ppt)

In [12]:
from cpm.optimisation import DifferentialEvolution, minimise, Fmin

lower = [1e-10, 1e-10]
upper = [1, 1]
bounds = list(zip(lower, upper))

In [ ]:
Fit = Fmin(
    model=wrap,
    data=experiment,
    initial_guess=[0.32, 0.788],
    minimisation=minimise.LogLikelihood.continuous,  # currently, this is the only working metric
    parallel=True
)

Fit.optimise()

AttributeError: Can't pickle local object 'Fmin.optimise.<locals>.__task'

   policy_0  values_0  values_1  values_2  values_3  dependent_0  ppt
0       0.0       0.0       0.5       0.5       0.0          0.0    0
0       0.0       0.0       0.5       0.5       0.0          0.0    0
0       1.0       0.0       0.5       0.5       0.0          1.0    0
0       0.0       0.0       0.5       0.5       0.0          0.0    0
0       1.0       0.0       0.5       0.5       0.0          1.0    0
0       1.0       0.0       0.5       0.5       0.0          1.0    0
0       0.0       0.0       0.5       0.5       0.0          0.0    0
0       1.0       0.0       0.5       0.5       0.0          1.0    0
0       0.0       0.0       0.5       0.5       0.0          0.0    0
0       1.0       0.0       0.5       0.5       0.0          1.0    0
   policy_0  values_0  values_1  values_2  values_3  dependent_0  ppt
0   0.00000       0.0  0.050000  0.050000       0.0      0.00000    0
0   0.00000       0.0  0.050000  0.050000       0.0      0.00000    0
0   0.10000       0.

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/spawn.py", line 131, in _main
    prepare(preparation_data)
  File "/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/spawn.py", line 246, in prepare
    _fixup_main_from_path(data['init_main_from_path'])
  File "/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/spawn.py", line 297, in _fixup_main_from_path
    main_content = runpy.run_path(main_path,
                   ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen runpy>", line 28

   policy_0  values_0  values_1  values_2  values_3  dependent_0  ppt
0       0.0       0.0       0.5       0.5       0.0          0.0    0
0       0.0       0.0       0.5       0.5       0.0          0.0    0
0       1.0       0.0       0.5       0.5       0.0          1.0    0
0       0.0       0.0       0.5       0.5       0.0          0.0    0
0       1.0       0.0       0.5       0.5       0.0          1.0    0
0       1.0       0.0       0.5       0.5       0.0          1.0    0
0       0.0       0.0       0.5       0.5       0.0          0.0    0
0       1.0       0.0       0.5       0.5       0.0          1.0    0
0       0.0       0.0       0.5       0.5       0.0          0.0    0
0       1.0       0.0       0.5       0.5       0.0          1.0    0
   policy_0  values_0  values_1  values_2  values_3  dependent_0  ppt
0       0.0       0.0       0.5       0.5       0.0          0.0    0
0       0.0       0.0       0.5       0.5       0.0          0.0    0
0       1.0       0.

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/spawn.py", line 131, in _main
    prepare(preparation_data)
  File "/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/spawn.py", line 246, in prepare
    _fixup_main_from_path(data['init_main_from_path'])
  File "/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/spawn.py", line 297, in _fixup_main_from_path
    main_content = runpy.run_path(main_path,
                   ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen runpy>", line 28

...

   policy_0  values_0  values_1  values_2  values_3  dependent_0  ppt
0       0.0       0.0       0.5       0.5       0.0          0.0    0
0       0.0       0.0       0.5       0.5       0.0          0.0    0
0       1.0       0.0       0.5       0.5       0.0          1.0    0
0       0.0       0.0       0.5       0.5       0.0          0.0    0
0       1.0       0.0       0.5       0.5       0.0          1.0    0
0       1.0       0.0       0.5       0.5       0.0          1.0    0
0       0.0       0.0       0.5       0.5       0.0          0.0    0
0       1.0       0.0       0.5       0.5       0.0          1.0    0
0       0.0       0.0       0.5       0.5       0.0          0.0    0
0       1.0       0.0       0.5       0.5       0.0          1.0    0
   policy_0  values_0  values_1  values_2  values_3  dependent_0  ppt
0       0.0       0.0       0.5       0.5       0.0          0.0    0
0       0.0       0.0       0.5       0.5       0.0          0.0    0
0       1.0       0.

hase.

        This probably means that you are not using fork to start your
        child processes and you have forgotten to use the proper idiom
        in the main module:

            if __name__ == '__main__':
                freeze_support()
                ...

        The "freeze_support()" line can be omitted if the program
        is not going to be frozen to produce an executable.

        To fix this issue, refer to the "Safe importing of main module"
        section in https://docs.python.org/3/library/multiprocessing.html
        


   policy_0  values_0  values_1  values_2  values_3  dependent_0  ppt
0       0.0       0.0       0.5       0.5       0.0          0.0    0
0       0.0       0.0       0.5       0.5       0.0          0.0    0
0       1.0       0.0       0.5       0.5       0.0          1.0    0
0       0.0       0.0       0.5       0.5       0.0          0.0    0
0       1.0       0.0       0.5       0.5       0.0          1.0    0
0       1.0       0.0       0.5       0.5       0.0          1.0    0
0       0.0       0.0       0.5       0.5       0.0          0.0    0
0       1.0       0.0       0.5       0.5       0.0          1.0    0
0       0.0       0.0       0.5       0.5       0.0          0.0    0
0       1.0       0.0       0.5       0.5       0.0          1.0    0
   policy_0  values_0  values_1  values_2  values_3  dependent_0  ppt
0   0.00000       0.0  0.050000  0.050000       0.0      0.00000    0
0   0.00000       0.0  0.050000  0.050000       0.0      0.00000    0
0   0.10000       0.

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/spawn.py", line 131, in _main
    prepare(preparation_data)
  File "/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/spawn.py", line 246, in prepare
    _fixup_main_from_path(data['init_main_from_path'])
  File "/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/spawn.py", line 297, in _fixup_main_from_path
    main_content = runpy.run_path(main_path,
                   ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen runpy>", line 28

   policy_0  values_0  values_1  values_2  values_3  dependent_0  ppt
0       0.0       0.0       0.5       0.5       0.0          0.0    0
0       0.0       0.0       0.5       0.5       0.0          0.0    0
0       1.0       0.0       0.5       0.5       0.0          1.0    0
0       0.0       0.0       0.5       0.5       0.0          0.0    0
0       1.0       0.0       0.5       0.5       0.0          1.0    0
0       1.0       0.0       0.5       0.5       0.0          1.0    0
0       0.0       0.0       0.5       0.5       0.0          0.0    0
0       1.0       0.0       0.5       0.5       0.0          1.0    0
0       0.0       0.0       0.5       0.5       0.0          0.0    0
0       1.0       0.0       0.5       0.5       0.0          1.0    0
